In [2]:
from epymorph.data_shape import Shapes
from epymorph.geo.adrio import adrio_maker_library
from epymorph.geo.dynamic import DynamicGeo
from epymorph.geo.spec import DynamicGeoSpec, Year
from epymorph.geography.us_census import CountyScope, StateScope
from epymorph.simulation import geo_attrib


spec = DynamicGeoSpec(
    attributes=[
        geo_attrib('label', str, Shapes.N),
        geo_attrib('population', int, Shapes.N),
        geo_attrib("covid_cases_per_100k", int, Shapes.N),
        geo_attrib("covid_hospitalizations_per_100k", int, Shapes.N),
        geo_attrib("covid_hospitalization_avg", float, Shapes.N),
        geo_attrib("covid_hospitalization_sum", int, Shapes.N),
        geo_attrib("influenza_hospitalization_avg", float, Shapes.N),
        geo_attrib("influenza_hospitalization_sum", int, Shapes.N),
        geo_attrib("full_covid_vaccinations", int, Shapes.N),
        geo_attrib("one_dose_covid_vaccinations", int, Shapes.N),
        geo_attrib("covid_booster_doses", int, Shapes.N),
        geo_attrib("covid_deaths", int, Shapes.N)
    ],
    time_period=Year(2022),
    # scope=CountyScope.in_counties(['04007', '04013']),
    # scope=CountyScope.in_states(['04', '08']),
    scope=StateScope.in_states(['04', '08']),
    source={
        'label': 'Census:name',
        'population': 'Census',
        "covid_cases_per_100k": "CDC",
        "covid_hospitalizations_per_100k": "CDC",
        "covid_hospitalization_avg": "CDC",
        "covid_hospitalization_sum": "CDC",
        "influenza_hospitalization_avg": "CDC",
        "influenza_hospitalization_sum": "CDC",
        "full_covid_vaccinations": "CDC",
        "one_dose_covid_vaccinations": "CDC",
        "covid_booster_doses": "CDC",
        "covid_deaths": "CDC"
    }
)

geo = DynamicGeo.from_library(spec, adrio_maker_library)

geo.fetch_all()
# geo['covid_cases_100k']
# geo['covid_hospitalization_avg']
# geo['vaccinations']
# geo['covid_deaths']